In [219]:
import pandas as pd
import numpy as np
import re
import urllib.request
from bs4 import BeautifulSoup
import snscrape.modules.twitter as sntwitter
import csv
from selenium import webdriver
import time
from datetime import datetime, timedelta

pd.set_option("display.max_rows", 1000)

In [2]:
# scrape polling data
url = 'https://en.wikipedia.org/wiki/Opinion_polling_for_the_next_Australian_federal_election'
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page, "lxml")

In [3]:
tables = soup.find_all('table', class_='toccolours')
voter_intent_table = tables[0]
pref_pm_table = tables[1]

In [4]:
vi_date = []
vi_brand = []
vi_firm = []
vi_mode = []
vi_size = []

vi_lnp_primary = []
vi_alp_primary = []
vi_grn_primary = []
vi_onp_primary = []
vi_oth_primary = []
vi_und_primary = []

In [5]:
vi_data_list = [vi_brand, vi_firm, vi_mode, vi_size, vi_lnp_primary, vi_alp_primary, vi_grn_primary, 
                 vi_onp_primary, vi_oth_primary, vi_und_primary]

In [6]:
for row in voter_intent_table.findAll('tr'):
    dates = row.findAll('th')
    data = row.findAll('td')
    try:
        vi_date.append(dates[0].find(text=True))
    except:
        pass
    
    try:    
        for i in range(len(vi_data_list)):
            vi_data_list[i].append(data[i].find(text=True))
    except:
            pass

In [7]:
vi_date = [x.strip() for x in vi_date]
vi_date = vi_date[2:]

date_rm = ['27 February 2020', '4 February 2020', 'January 2020', '30 May 2019', '18 May 2019 election']
vi_date = [item for item in vi_date if item not in date_rm]

brand_rm = ['COVID-19 pandemic', 'Adam Bandt replaces Richard Di Natale as Greens leader', 'Australian bushfire season',
            'Anthony Albanese replaces Bill Shorten as Labor leader', '\n']
vi_brand = [item for item in vi_brand if item not in brand_rm]

# don't need last element for these ones
vi_mode = vi_mode[:-1]
vi_size = vi_size[:-1]
vi_lnp_primary = vi_lnp_primary[:-1]
vi_alp_primary = vi_alp_primary[:-1]
vi_grn_primary = vi_grn_primary[:-1]
vi_onp_primary = vi_onp_primary[:-1]
vi_oth_primary = vi_oth_primary[:-1]
vi_und_primary = vi_und_primary[:-1]

# strip new line characters
vi_mode = [x.strip() for x in vi_mode]
vi_size = [x.strip() for x in vi_size]
vi_lnp_primary = [x.strip() for x in vi_lnp_primary]
vi_alp_primary = [x.strip() for x in vi_alp_primary]
vi_grn_primary = [x.strip() for x in vi_grn_primary]
vi_onp_primary = [x.strip() for x in vi_onp_primary]
vi_oth_primary = [x.strip() for x in vi_oth_primary]
vi_und_primary = [x.strip() for x in vi_und_primary]

In [8]:
vi_data = pd.DataFrame({'date': vi_date, 'brand': vi_brand, 'mode': vi_mode, 'size': vi_size,
                       'lnp': vi_lnp_primary, 'alp': vi_alp_primary})

In [10]:
vi_data.head()

,date,brand,mode,size,lnp,alp
0,24–27 March 2021,Newspoll,Online,1517,40%,38%
1,"6–7, 13–14 March 2021",Morgan Poll,Telephone/Online,2747,41%,34.5%
2,10–13 March 2021,Newspoll,Online,1521,39%,39%
3,17–20 February 2021,Newspoll,Online,1504,42%,37%
4,"6–7, 13–14 February 2021",Morgan Poll,Telephone/Online,2786,40%,34.5%


In [25]:
pp_date = []
pp_brand = []
pp_firm = []
pp_mode = []
pp_size = []
pp_mor = []
pp_alb = []
pp_dnk = []
_ = []
mor_sat = []
mor_dis = []
mor_dnk = []
alb_sat = []
alb_dis = []
alb_dkn = []

In [26]:
pp_data_list = [pp_brand, pp_firm, pp_mode, pp_size, pp_mor, pp_alb, pp_dnk, _, mor_sat, mor_dis,
                mor_dnk, alb_sat, alb_dis, alb_dkn]

In [27]:
for row in pref_pm_table.findAll('tr'):
    dates = row.findAll('th')
    data = row.findAll('td')
    try:
        pp_date.append(dates[0].find(text=True))
    except:
        pass
    
    try:    
        for i in range(len(pp_data_list)):
            pp_data_list[i].append(data[i].find(text=True))
    except:
            pass

In [28]:
pp_date = [x.strip() for x in pp_date]
pp_mode = [x.strip() for x in pp_mode]
pp_size = [x.strip() for x in pp_size]
pp_mor = [x.strip() for x in pp_mor]
pp_alb = [x.strip() for x in pp_alb]
pp_dnk = [x.strip() for x in pp_dnk]
mor_sat = [x.strip() for x in mor_sat]
mor_dis = [x.strip() for x in mor_dis]
mor_dnk = [x.strip() for x in mor_dnk]
alb_sat = [x.strip() for x in alb_sat]
alb_dis = [x.strip() for x in alb_dis]
alb_dkn = [x.strip() for x in alb_dkn]

pp_date = pp_date[2:]

In [122]:
# convert date to month-year
# take average of polls for a month
# predict that month's twitter to result
# then can use for real time analysis

In [123]:
# just focus on pp data

In [137]:
pp_data = pd.DataFrame({'date': pp_date, 'brand': pp_brand, 'mode': pp_mode, 'size': pp_size,
                       'mor_sat': mor_sat, 'mor_dis': mor_dis})

months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 
          'September', 'October', 'November', 'December']
years = ['2019', '2020', '2021']

pp_data['month'] = pp_data.date.str.extract('({})'.format('|'.join(months)), expand=False).str.title().fillna('')
pp_data['year'] = pp_data.date.str.extract('({})'.format('|'.join(years)), expand=False).fillna('')
# one year was missing
pp_data.loc[1, 'year'] = '2021'
pp_data['date'] = pp_data['month'] + " " + pp_data['year']
del pp_data['month']
del pp_data['year']

# make date a datetime
pp_data['date'] = pd.to_datetime(pp_data['date'])

# remove percent symbols and make numeric cols numeric
pp_data['mor_sat'] = pp_data['mor_sat'].str.replace('%', "")
pp_data['mor_dis'] = pp_data['mor_dis'].str.replace('%', "")
pp_data['mor_sat'] = pp_data['mor_sat'].str.replace('n/a', '')
pp_data['mor_dis'] = pp_data['mor_dis'].str.replace('n/a', '')
pp_data['size'] = pd.to_numeric(pp_data['size'], errors='coerce')
pp_data['mor_sat'] = pd.to_numeric(pp_data['mor_sat'], errors='coerce')
pp_data['mor_dis'] = pd.to_numeric(pp_data['mor_dis'], errors='coerce')

In [138]:
pp_data.head()

,date,brand,mode,size,mor_sat,mor_dis
0,2021-03-01,Newspoll,Online,1517.0,55.0,40.0
1,2021-03-01,Essential,Online,1100.0,57.0,35.0
2,2021-03-01,Newspoll,Online,1521.0,62.0,34.0
3,2021-02-01,Newspoll,Online,1504.0,64.0,32.0
4,2021-01-01,Newspoll,Online,1512.0,63.0,33.0


In [141]:
pp_mean = pp_data.groupby('date').mean()

In [142]:
pp_data.to_csv('pp_data.csv')
pp_mean.to_csv('pp_mean.csv')

In [ ]:
# scrape tweets
# https://github.com/Mottl/GetOldTweets3/issues/98
# Create a file to append data to
tweets_csv = open('tweets.csv', 'a', newline='', encoding='utf8')

#Use csv writer
csv_writer = csv.writer(tweets_csv)
csv_writer.writerow(['id','date','tweet',]) 

for i, tweet in enumerate(sntwitter.TwitterSearchScraper('scott morrison + since:2019-06-01 until:2021-03-31 -filter:links -filter:replies').get_items()):  
        try:
            csv_writer.writerow([tweet.id, tweet.date, tweet.content])
        except:
            pass
        
tweets_csv.close()

In [192]:
# Scrape google news headlines search scott morrison from 2019 to 2021
# List to save results
headlines = []

# Search term
url = "https://www.google.com/search?q=scott+morrison&rlz=1C1CHBF_en-GBAU925AU926&tbs=cdr:1,cd_min:2019,cd_max:2021&tbm=nws&sxsrf=ALeKk03SVeb5dZQZk3ek-_Z4QIqSOr823A:1617399600351&ei=MI9nYND0FOaU4-EPwauZsAQ&start=0&sa=N&ved=0ahUKEwjQzI2Fw-DvAhVmyjgGHcFVBkY4MhDy0wMIhAE&biw=1920&bih=969&dpr=1"
driver = webdriver.Chrome()
driver.get(url)

# Scrape first page results
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") # scroll to end of page
time.sleep(5) # wait
# Get from results
results = driver.find_elements_by_class_name("dbsr")
# Append text to headlines list
for i in results:
    headlines.append(i.text)
    
# Click to next page
driver.find_element_by_link_text("Next").click()
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(5)

# There are 16 pages of results
#pages = 16

# scrape intermediate pages
for i in range(14):
    results = driver.find_elements_by_class_name("dbsr")
    for i in results:
        headlines.append(i.text)
    time.sleep(5)
    driver.find_element_by_link_text("Next").click()
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
# srape last page
results = driver.find_elements_by_class_name("dbsr")
for i in results:
    headlines.append(i.text)

In [194]:
headlines[-2]

"The New Daily\nAlan Kohler: Scott Morrison should be thanking the heavens for the economy\nPretty soon the economy is going to elevate Prime Minister Scott Morrison out of the hole he's dug for himself over the past few weeks through seriously pissing ...\n2 days ago"

In [195]:
len(headlines)

160

In [203]:
paper = []
text = []
date = []

for headline in headlines:
    # Name of paper
    paper.append(''.join(headline.split('\n')[0]))
    # Headline
    text.append(''.join(headline.split('\n')[1:-2]))
    # Date
    date.append(''.join(headline.split('\n')[-1]))

In [373]:
news = pd.DataFrame({'date': date, 'headline': text})

In [374]:
dates_ok = news.loc[:128]
dates_fix = news.loc[129:]

In [375]:
# Remove any extra text in front of date - interferes with conversion
dates_ok['date'] = dates_ok['date'].str.extract(r'(\d+.+$)')

# convert to datetime
dates_ok['date2'] = pd.to_datetime(dates_ok['date'], errors ='coerce')

# Filter into ok and not ok
dates_not_ok = dates_ok[dates_ok['date2'].isnull()]
dates_ok = dates_ok[dates_ok['date2'].notnull()]
del dates_not_ok['date2']
dates_fix = pd.concat([dates_fix, dates_not_ok])
dates_ok['date'] = dates_ok['date2']
del dates_ok['date2']

<ipython-input-375-2f6e301a45c7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dates_ok['date'] = dates_ok['date'].str.extract(r'(\d+.+$)')
<ipython-input-375-2f6e301a45c7>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dates_ok['date2'] = pd.to_datetime(dates_ok['date'], errors ='coerce')


In [376]:
dates_fix['number'] = dates_fix.loc[:, 'date'].str.split().str[0]
dates_fix['period'] = dates_fix.loc[:, 'date'].str.split().str[1]
dates_fix['period'] = dates_fix.loc[:,'period'].apply(lambda x: x[:-1] if x.endswith('s') else x)

# Function to get number of days
def num_days(row):
    if row['period'] == 'day':
        return int(row['number'])
    elif row['period'] == 'week':
        return int(row['number'])*7
    elif row['period'] == 'hour':
        return 1
    elif row['period'] == 'month':
        return int(row['number'])*30
    
dates_fix['days'] = dates_fix.apply(lambda row: num_days(row), axis=1)
today = datetime.now()    
dates_fix['date'] = today - dates_fix['days'].map(timedelta)
dates_fix['date'] = dates_fix['date'].dt.date
dates_ok['date'] = dates_ok['date'].dt.date

del dates_fix['number']
del dates_fix['period']
del dates_fix['days']

news = pd.concat([dates_ok, dates_fix])
news.to_csv('google_news_headlines.csv', index=False)

In [382]:
news.sort_values(by='date', ascending=False)

,date,headline
147,2021-04-02,"‘Too late for me, Scotty’: Readers respond to ..."
150,2021-04-02,Is Scott Morrison safe until the election? – A...
135,2021-04-02,"Easter a time of ‘love, ‘forgiveness’: Scott M..."
154,2021-04-02,Prime Minister Scott Morrison praises Australi...
157,2021-04-02,Scott Morrison is facing three major problems ...
145,2021-04-02,Scott Morrison must beware of Churchill syndrome
144,2021-04-01,Scott Morrison tight-lipped on moving SA’s Col...
137,2021-04-01,States hit back at Morrison government amid gr...
156,2021-04-01,COVID vaccine Australia: vaccine rollout will ...
140,2021-04-01,States hit back at Morrison government amid gr...


In [ ]:
# first line is the newspaper - remove
# last line is the date - put into a new column
# remove "this article is more than 1 year old" and similar